In [1]:
import os
import sys
sys.path.append('../../kl_tools/')
import numpy as np
from astropy.units import Unit
import matplotlib.pyplot as plt
import galsim as gs
import galsim.roman as roman

import basis
import cube
import intensity
import likelihood
import mcmc
import parameters
import priors
import utils
import velocity

# new modules
#from spectrum import Spectrum
#from MockObs import Mock
%matplotlib inline

In [2]:
theta_pars = {'g1':0.0, 'g2':0.0, 'theta_int': 0., 'sini':0.5,'v0':0,'vcirc':200,'rscale':0.5}
pars = {
    'priors':{
        'g1': priors.GaussPrior(0., 0.1),
        'g2': priors.GaussPrior(0., 0.1),
        'theta_int': priors.UniformPrior(-np.pi/2., np.pi/2.),
        'sini': priors.UniformPrior(0., 1.),
        'v0': priors.GaussPrior(0, 20),
        'vcirc': priors.GaussPrior(200, 10),
        'rscale': priors.UniformPrior(0, 10),
    },
    'meta':{
        'Nx':64,
        'Ny':64,
        'scale':0.1,# arcsec
        'lambda_range':[1200, 1400],
        'lambda_res': 1,# nm
    },
    'intensity':{
        'type':'inclined_exp',
        'hlr':0.5,# arcsec
    },
    'velocity':{
        'v_unit':Unit('km / s'),
        'r_unit':Unit('arcsec'),
    },
    'SED':{
        'template':'../../data/Simulation/GSB2.spec',
        'wave_type': 'Ang',
        'flux_type': 'flambda',
        'z': 1.0,
        'wave_range': (500, 3000), # nm
        # obs-frame continuum normalization (nm, erg/s/cm2/nm)
        'obs_cont_norm': (1400, 1e-17),
        # a dict of line names and obs-frame flux values (erg/s/cm2)
        'lines': {'Halpha': 1e-15, 'OII':[1e-15, 1.2e-15], 'OIII':[1e-15, 1.2e-15]},
        # intrinsic linewidth in nm
        'line_sigma_int': {'Halpha': 4, 'OII':[2,2], 'OIII':[3,3]},
    },
    # observation related parameters
    # Note that from the same galaxy and shear model, we can derive 
    # multiple types of data,
    # e.g. `photometry`, `slit_spectroscopy`, `grism` and/or `IFU`.
    # Thus 'observations' is a list of dictionaries that specify 
    # observation-wise parameters.
    # TODO: maybe think about more flexible way to describe dispersion
    'observations':[

        # Roman WFI/GRISM observation, roll angle 1
        {'inst_name':'Roman/WFI',
         'type':'grism',
         'bandpass':'../../data/Bandpass/Roman/WFI_Grism_1stOrder.dat',
         'Nx': 64,# number of pixels
         'Ny': 64,
         'pixel_scale': 0.11,# arcsec
         'R_spec':461,# at 1 micron
         'offset':-1210.22,# pix
         # can be 'none'/'airy'/'moffat'
         # 'kolmogorov'/'vonkarman'/'opticalpsf' to be implemented.
         'psf_type':'airy',
         # pass the needed params to build PSF model here
         # in case of airy, we don't need any params
         'psf_kwargs':{'fwhm':0.13},# arcsec
         'disp_ang':0.,# radian
         'diameter':240,# cm
         'exp_time':600.,# seconds
         'gain':1.,
         #'noise':{'type':'ccd','sky_level':0.65*1.2,'read_noise':8.5},
         },
        # Roman WFI/GRISM observation, roll angle 2
        {'inst_name':'Roman/WFI',
         'type':'grism',
         'bandpass':'../../data/Bandpass/Roman/WFI_Grism_1stOrder.dat',
         'Nx': 64,# number of pixels
         'Ny': 64,
         'pixel_scale': 0.11,# arcsec
         'R_spec':461,# at 1 micron
         'offset':-1210.22,
         'psf_type':'airy',
         'psf_kwargs':{'fwhm':0.13},# arcsec
         'disp_ang':np.pi/2.,# radian
         'diameter':240,# cm
         'exp_time':600.,# seconds
         'gain':1.,
         #'noise':{'type':'ccd','sky_level':0.65*1.2,'read_noise':8.5},
         },
        # Roman WFI/Photometry image observation, H band
        {'inst_name':'Roman/WFI',
         'type':'photometry',
         'bandpass':'../../data/Bandpass/Roman/WFI_F129.dat',
         'Nx': 32,# number of pixels
         'Ny': 32,
         'pixel_scale': 0.11,# arcsec
         'psf_type':'airy',
         'psf_kwargs':{'fwhm':0.13},# arcsec
         'diameter':240,# cm
         'exp_time':600.,# seconds
         'gain':1.,
         #'noise':{'type':'ccd','sky_level':0.65*1.2,'read_noise':8.5},
         },
    ],
    'use_numba': False,
}

In [3]:
### Build fiducial model cube
fiducial_theta = {
    'g1':0.0, 
    'g2':0.0, 
    'theta_int': 0., 
    'sini':0.5,
    'v0':0,
    'vcirc':200,
    'rscale':0.5,
}
halpha = 656.28 # nm
R = 461.
z = 0.3
fiducial_meta = {
    # meta parameters
    'Nx': 30, # pixels, for 3d model cube
    'Ny': 30, # pixels
    'scale': 0.1, # pixel scale for model cube
    'sed_start': 655,
    'sed_end': 657.5,
    'sed_resolution': 0.025,
    # intensity profile
    'intensity': {
        # For this test, use truth info
        'type': 'inclined_exp',
        'flux': 1e5, # counts
        'hlr': 5, # pixels
        # 'type': 'basis',
        # 'basis_type': 'shapelets',
        # 'basis_type': 'sersiclets',
        # 'basis_type': 'exp_shapelets',
        # 'basis_kwargs': {
        #     'Nmax': 7,
        #     # 'plane': 'disk',
        #     'plane': 'obs',
        #     'beta': 0.35,
        #     'index': 1,
        #     'b': 1,
        #     }
    },
    # kinematics
    'v_unit': Unit('km / s'),
    'r_unit': Unit('arcsec'),
    # SED
    'z': z,
    'true_flux': 1e5, # counts
    'true_hlr': 5, # pixels
    'line_std': halpha * (1.+z) / R, # emission line SED std; nm
    'line_value': 656.28, # emission line SED std; nm
    'line_unit': Unit('nm'),
    'sed_unit': Unit('nm'),
    # observation
    'pix_scale': 0.13, # arcsec / pixel
    'cov_sigma': 3., # pixel counts; dummy value
    'bandpass_throughput': '.2',
    'bandpass_unit': 'nm',
    'bandpass_zp': 30,
    'spec_resolution': R,
    # priors
    'priors': {
        'g1': priors.GaussPrior(0., 0.3, clip_sigmas=2),
        'g2': priors.GaussPrior(0., 0.3, clip_sigmas=2),
        # 'theta_int': priors.UniformPrior(0., np.pi),
        'theta_int': priors.UniformPrior(0., np.pi),
        # 'theta_int': priors.UniformPrior(np.pi/3, np.pi),
        'sini': priors.UniformPrior(0., 1.),
        'v0': priors.UniformPrior(0, 20),
        'vcirc': priors.GaussPrior(200, 20, zero_boundary='positive'),# clip_sigmas=2),
        # 'vcirc': priors.GaussPrior(188, 2.5, zero_boundary='positive', clip_sigmas=2),
        # 'vcirc': priors.UniformPrior(190, 210),
        'rscale': priors.UniformPrior(0, 10),
    },
    'psf': {
        'test': None
    },
    'sed': {
        'test': None
    },
    
    # 'marginalize_intensity': True,
    # 'psf': gs.Gaussian(fwhm=3), # fwhm in pixels
    'use_numba': False,
}

In [5]:
pars = parameters.Pars(list(fiducial_theta.keys()), fiducial_meta)
# dimension init 
li, le, dl = 655.8, 656.8, 0.1
lambdas = [(l, l+dl) for l in np.arange(li, le, dl)]
Nx, Ny = pars.meta['Nx'], pars.meta['Ny']
Nspec = len(lambdas)
shape = (Nspec, Nx, Ny)
# trivial bandpass
bandpasses = []
for l1, l2 in lambdas:
    bandpasses.append(gs.Bandpass(
        pars.meta['bandpass_throughput'], pars.meta['bandpass_unit'], 
        blue_limit=l1, red_limit=l2, zeropoint=pars.meta['bandpass_zp']
        ))
# empty datavector
null_dv = cube.GrismData(shape=shape, bandpasses=bandpasses, pix_scale=pars.meta['scale'], 
                         pars=pars.meta.pars)
# init likelihood
like = likelihood.LogLikelihood(pars, null_dv)
# 